In [ ]:
!pip install praw
!pip install asyncpraw
!pip install langdetect

In [ ]:
import pandas as pd
from datasets import load_dataset

import os
import praw
from time import sleep

from langdetect import detect, DetectorFactory

In [ ]:
# Parameters (For HuggingFace Dataset)
TARGET_PER_LABEL = 2000
MIN_LENGTH = 100             # ignore very short posts
MAX_LENGTH = 800             # max characters per post

# Part 1: Dataset Collection (Hugging Face)

**Dataset Source:** [reddit_mental_health_posts](https://huggingface.co/datasets/solomonk/reddit_mental_health_posts/viewer/default/train?f%5Bsubreddit%5D%5Bvalue%5D=%27aspergers%27&views%5B%5D=train)

For this project, we will curate a balanced subset of **2,000 posts** across the following 4 mental health categories:

1. **PTSD (Post-Traumatic Stress Disorder)**
2. **OCD (Obsessive–Compulsive Disorder)**
3. **ADHD (Attention-Deficit/Hyperactivity Disorder)**
4. **Depression**

In [ ]:
data_hf = load_dataset("solomonk/reddit_mental_health_posts")

In [ ]:
data_hf

In [ ]:
df = data_hf['train'].to_pandas()
df

In [ ]:
df = df.rename(columns = {"id" : "post_id"})
df = df[['post_id', 'subreddit', 'title', 'body']]
df

In [ ]:
df = df.drop_duplicates(subset="body").reset_index(drop=True)
df = df[df['body'] != '[deleted]']
df = df.dropna()
df['subreddit'].value_counts()

In [ ]:
df = df[df['subreddit'] != 'aspergers']

In [ ]:
df = df[df['body'].astype(str).apply(len).between(MIN_LENGTH, MAX_LENGTH)]
df['subreddit'].value_counts()

In [ ]:
df_hf = pd.DataFrame()

for label in df['subreddit'].unique():
  df_hf = pd.concat([df_hf, df[df['subreddit'] == label].sample(n=TARGET_PER_LABEL, random_state=42)])

df_hf = df_hf.reset_index(drop=True)


In [ ]:
df_hf['subreddit'].value_counts()

In [ ]:
subreddit_to_label = {
    "ADHD" : "ADHD",
    "depression" : "Depression",
    "OCD" : "OCD",
    "ptsd" : "PTSD"
}

df_hf['label'] = df_hf['subreddit'].map(subreddit_to_label)

df_hf

In [ ]:
df_hf.to_csv('huggingFace_mental_health_dataset.csv', index=False)

In [ ]:
from google.colab import files
files.download('huggingFace_mental_health_dataset.csv')

# Part 2: Collecting Posts from Reddit  

In this stage, we will gather posts corresponding to **4 additional categories** to broaden the dataset:  

1. **Anxiety**  
2. **Addiction**  
3. **Suicidal Ideation**  
4. **Normal (Control Group)**

In [ ]:
# Parameters (for Reddit posts)
TARGET_PER_LABEL = 2200
MAX_LENGTH = 800             # max characters per post
SLEEP_TIME = 0.2             # seconds between requests to avoid rate limit
DetectorFactory.seed = 0  # for consistent results

In [ ]:
df_hf = pd.read_csv('/content/huggingFace_mental_health_dataset.csv')
save_file = "reddit_mental_health_dataset.csv"

if os.path.exists(save_file):
    df_existing = pd.read_csv(save_file)
    data_reddit = df_existing.to_dict("records") + df_hf.to_dict("records")
    seen_texts = set(df_existing["body"].tolist()) | set(df_hf["body"].tolist())  # avoid duplicates
    print(f"🔄 Resuming... already have {len(df_existing)} posts")
else:
    data_reddit = df_hf.to_dict("records")
    seen_texts = set(df_hf["body"].tolist())

In [ ]:
# 1. Setup Reddit API
reddit = praw.Reddit(client_id="YOUR_CLIENT_ID",
                     client_secret="YOUR_CLIENT_SECRET",
                     user_agent="YOUR_USER_AGENT")

In [ ]:
# 2. Define rest of subreddits

mental_subreddits = {
    "Anxiety": {
        "sub_list": ["anxiety", "socialanxiety", "PanicAttack", "HealthAnxiety"],
        "MIN_LENGTH": 80
    },
    "Suicidal": {
        "sub_list": ["SuicideWatch", "Suicidal_Thoughts", "SuicideBereavement"],
        "MIN_LENGTH": 50
    },
    "Addiction": {
        "sub_list": ["StopSmoking", "addiction", "StopDrinking", "stopgaming"],
        "MIN_LENGTH": 80
    },
    "Normal": {
        "sub_list": ["happy", "CasualConversation", "wholesomememes", "LifeProTips"],
        "MIN_LENGTH": 50
    }
}

In [ ]:
def is_english(text):
    try:
        return detect(text) == "en"
    except:
        return False

In [ ]:
def fetch_posts_for_label(label, subreddits, MIN_LENGTH):

    for subreddit_name in subreddits:
        try:
            subreddit = reddit.subreddit(subreddit_name)
            print(f"\nFetching '{label}' posts from r/{subreddit_name}...")

            for post in subreddit.new(limit=None):
                title = post.title.strip()
                body = post.selftext.strip()

                if len(body) < MIN_LENGTH or len(body) > MAX_LENGTH or body in seen_texts:
                    continue
                if not is_english(body):
                    continue

                seen_texts.add(body)

                data_reddit.append({
                    "post_id": post.id,
                    "subreddit": subreddit_name,
                    "title": title,
                    "body": body,
                    "label": label
                })

                sleep(SLEEP_TIME)

            # Save after each subreddit
            df_reddit = pd.DataFrame(data_reddit)
            df_reddit.to_csv(save_file, index=False)
            print(f" Progress saved: {len(df_reddit)} posts so far")

            current_label_count = sum(1 for d in data_reddit if d["label"] == label)
            if current_label_count >= TARGET_PER_LABEL:
              return

        except Exception as e:
            print(f" Could not fetch from r/{subreddit_name}: {e}")


In [ ]:
# fetch call
for label, subreddits in mental_subreddits.items():
    fetch_posts_for_label(label, subreddits["sub_list"], subreddits["MIN_LENGTH"])


In [ ]:
df_reddit = pd.DataFrame(data_reddit)
df_reddit = df_reddit.drop_duplicates(subset="body").reset_index(drop=True)
df_reddit['label'].value_counts()

In [ ]:
BALANCE_SIZE = 2000

balanced_df = (
    df_reddit
    .groupby("label", group_keys=False)   # group by label
    .apply(lambda x: x.sample(n=min(BALANCE_SIZE, len(x)), random_state=42))  # sample within each class
    .reset_index(drop=True)
)

balanced_df["label"].value_counts()

In [ ]:
from google.colab import files
balanced_df.to_csv(save_file, index=False)
files.download(save_file)

# Part 3: Dataset Creation for GitHub Repository  

- **Post IDs will be removed** to avoid any privacy issues.  
- Only essential columns (e.g., title, subreddit, body, label) will be retained.  


In [ ]:
df = pd.read_csv('/content/reddit_mental_health_dataset.csv')

In [ ]:
df = df.drop(columns=['post_id'])
df.to_csv('reddit_mental_health_dataset.csv', index=False)